In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Documents\\Data Science\\Projects\\Chicken_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Documents\\Data Science\\Projects\\Chicken_Disease_Classification'

In [5]:
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

OSError: No file or directory found at artifacts/training/model.h5

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    test_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config.training.trained_model_path,
            test_data=self.config.data_ingestion.test_data_dir,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [9]:
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _test_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.test_data,
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._test_generator()  # Use the modified method
        self.score = model.evaluate(self.test_generator, return_dict =True)

    def save_score(self, filename="scores.json", directory="."):
        if self.score is not None:
            # scores = {"loss": self.score[0], "accuracy": self.score[1]}
            scores = self.score
            save_json(path=Path(directory) / filename, data=scores)

In [11]:
try:
    config = ConfigurationManager()
    val_config = config.get_evaluation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-12-03 22:13:30,563: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-03 22:13:30,569: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-03 22:13:30,572: INFO: common: created directory at: artifacts]



[2023-12-03 22:13:31,178: WARNING: module_wrapper: From c:\Users\DELL\.conda\envs\chicken\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]

[2023-12-03 22:13:31,413: WARNING: module_wrapper: From c:\Users\DELL\.conda\envs\chicken\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
Found 120 images belonging to 2 classes.

[2023-12-03 22:13:33,820: WARNING: module_wrapper: From c:\Users\DELL\.conda\envs\chicken\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
8/8 [==============================] - 33s 4s/step - loss: 0.5041 - accuracy: 0.9667 - precision_1: 0.9667 - recall_1: 0.9667 - auc_2: 0.9732
[2023-12-03 22:14:06,633: INFO: common: json file saved at: